## Which Offers Better Returns: Bitcoin's Volatility or the S&P 500's Consistency?

### Objective



In [43]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [44]:
# Import data and convert 'Date' value
btc_data = pd.read_csv('./Data/btc_data_aligned_with_sp500.csv', encoding='cp1252')
sp500_data = pd.read_csv('./Data/sp500_data_aligned_with_btc_data.csv', encoding='cp1252')

btc_data['Date'] = pd.to_datetime(btc_data['Date'])
sp500_data['Date'] = pd.to_datetime(sp500_data['Date'])

# Rename the 'Close' columns for clarity
btc_data.rename(columns={"Close": "BTC_Price"}, inplace=True)
sp500_data.rename(columns={"Close": "SP500_Price"}, inplace=True)

# Merging datasets
combined_data = pd.merge(btc_data[['Date', 'BTC_Price']], sp500_data[['Date', 'SP500_Price']], on='Date', how='inner')

# Ensure that Date is the index for the dataset
combined_data.set_index('Date', inplace=True)

combined_data

,BTC_Price,SP500_Price
Date,,
2014-09-18,424.440002,2011.359985
2014-09-19,394.795990,2010.400024
2014-09-22,402.152008,1994.290039
2014-09-23,435.790985,1982.770020
2014-09-24,423.204987,1998.300049
...,...,...
2024-04-24,64276.898438,5071.629883
2024-04-25,64481.707031,5048.419922
2024-04-26,63755.320313,5099.959961


### Function for calculate returns:

$$
\text{Returns (\%)} = \left( \frac{\text{Final Price} - \text{Start Price}}{\text{Start Price}} \right) \times 100
$$


In [45]:
# Function to calculate returns for a give frequency
def calculate_returns(prices, freq):
  '''
  Calculate percentage for a give frequency (e.g. 'YE', 'QE', 'M')
  
  Args: 
  prices (Dataframe): A dataframe with columns for a the asset prices.
  freq (str): Frequency ('YE' for yearly, 'QE' for quarterly, 'M' for monthly, 'W' for weekly) 
  '''
  
  return prices.resample(freq).apply(lambda x: (x.iloc[-1] - x.iloc[0]) / x.iloc[0] * 100) # Returns (%) = ((Final Price / Start Price) / Start Price) * 100)

# Filter data from 2017 onwards
filtered_data = combined_data.loc["2017-01-01":] # Starts from 2017-01-01 to the most recent date


##### Function for change the column's name (for better understanting)

In [46]:
def change_columns_name (data, freq):
  '''
  Change column names based on the provided frequency.
  
  Args: 
  data (DataFrame): Dataset to be changed. It MUST have columns "BTC_Price" and "SP500_Price".
  freq (str): Frequency ('YE', 'ME', 'QE', etc.).
  
  Returns: 
  DataFrame: The dataset with updated column names.
  '''
  
  # Define the suffix to use:
  if freq == 'YE': # Yearly
    suffix = "_Yearly_%"
  elif freq == 'QE': # Quarterly
    suffix = "_Quarterly_%"
  elif freq == 'ME': # Monthly
    suffix = "_Monthly_%"
  elif freq == 'W': # Weekly
    suffix = "_Weekly_%"
  else: # Default
    suffix = "_Returns_%"
    
  # Rename the columns based on the suffix
  data.rename(columns={
    "BTC_Price": f"BTC_Returns{suffix}",
    "SP500_Price": f"SP500_Returns{suffix}"
  }, inplace=True)
  
  return data
    


### Calculate Annual Returns

In [47]:
annual_returns = calculate_returns(filtered_data, 'YE')

# Change colmns names for better understanding of the output
change_columns_name(annual_returns, 'YE')

,BTC_Returns_Yearly_%,SP500_Returns_Yearly_%
Date,,
2017-12-31,1304.065822,18.415027
2018-12-31,-75.018852,-7.009394
2019-12-31,82.420803,28.714796
2020-12-31,315.172062,15.292907
2021-12-31,44.834761,28.793058
2022-12-31,-64.263326,-19.953051
2023-12-31,152.396656,24.729487
2024-12-31,32.449927,5.810034


### Calculate Quaterly Returns

In [48]:
quaterly_returns = calculate_returns(filtered_data, 'QE')
change_columns_name(quaterly_returns, 'QE')

,BTC_Returns_Quarterly_%,SP500_Returns_Quarterly_%
Date,,
2017-03-31,2.677621,4.645606
2017-06-30,116.892662,2.737355
2017-09-30,62.362415,3.719626
2017-12-31,232.391407,5.713054
2018-03-31,-52.171589,-2.037975
2018-06-30,-12.218019,5.286467
2018-09-30,0.452811,6.867985
2018-12-31,-43.203093,-14.283711
2019-03-31,3.929724,12.922948


### Calculate Monthly Returns

In [49]:
monthly_returns = calculate_returns(filtered_data, 'ME')
change_columns_name(monthly_returns, 'ME')

,BTC_Returns_Monthly_%,SP500_Returns_Monthly_%
Date,,
2017-01-31,-7.035269,0.931870
2017-02-28,19.306625,3.688879
2017-03-31,-12.328013,-1.387335
2017-04-30,15.096031,1.075099
2017-05-31,60.833564,0.982694
...,...,...
2024-01-31,-5.283520,2.167900
2024-02-29,42.071466,3.874291
2024-03-31,13.299545,2.282815


### Calculate Weekly Returns

In [50]:
weekly_returns = calculate_returns(filtered_data, 'W')
change_columns_name(weekly_returns, 'W')

,BTC_Returns_Weekly_%,SP500_Returns_Weekly_%
Date,,
2017-01-08,-13.569032,0.848155
2017-01-15,-8.733003,0.252986
2017-01-22,-1.422122,0.150808
2017-01-29,-0.137026,1.301871
2017-02-05,11.900278,0.724276
...,...,...
2024-04-07,-2.674965,-0.751943
2024-04-14,-6.192109,-1.518148
2024-04-21,0.658024,-1.868692
